In [1]:
import pandas as pd #for data frame
import numpy as np #maths
import matplotlib.pyplot as plt #graph
import seaborn as sns #graph
# import re
import time #time and date
# from collections import Counter
import itertools
from statsmodels.tsa.seasonal import seasonal_decompose #decomposesion'seasonality and trend'
from statsmodels.tsa.stattools import adfuller #'test'
from statsmodels.tsa.arima_model import ARIMA #algoritham 
# from statsmodels.tsa.ar_model import AutoReg
# from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.graphics.tsaplots import plot_acf
# from sklearn.metrics import mean_squared_error
# from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
sns.set()

In [2]:
path = "E:/Documents/Alagasan/PG/Projects/Main Project/Sales DB/"

In [3]:
df = pd.read_excel(path+"Retail sales.xlsx")

In [4]:
df1 = pd.read_excel(path+"Retail sales 2.xlsx")

In [5]:
df.shape

(1010640, 52)

In [6]:
df1.shape

(236344, 52)

In [7]:
dft = pd.concat([df,df1],axis=0)

In [8]:
dft.shape

(1246984, 52)

In [ ]:
df.describe()

In [ ]:
dft.describe(include='object')
# unique tells total type of data
# top tells top data which is repeated often
# feq tells number of top data

In [ ]:
dft = dft.sort_values(by=["Created Date"])#sorting data frame date wise

In [ ]:
imp_df = dft[["MRP","Unit Selling Price","Quantity",'Discount',
 'Sales Return/Refunded Qty',
 'Sales Return/Refunded Amount',
 'Sales Return/Refunded Tax',
 'Sales Returns/Total Refunds',
 'Items Sold',
 'Gross Sales',
 'Net Sales(Gross Sales - Sales Return/Refunded Amount - Discounted Amount)',
 'Net Tax Collected(Total Tax - Discounted Tax - Sales Return/Refunded Tax)',
 'Net Total(Net Sales + Net Tax Collected)']]

In [ ]:
plt.subplots(figsize=(25,25))
sns.heatmap(imp_df.corr(),annot=True)

In [ ]:
plt.subplots(figsize=(9,9))
dft.groupby("Location Name")["Gross Sales"].sum().sort_values(ascending=False).plot(kind="pie",autopct="%.2f")

In [ ]:
plt.subplots(figsize=(9,9))
dft.groupby("Employee Name")["Gross Sales"].sum().sort_values(ascending=False).head(10).plot(kind="pie",autopct="%.2f")

In [ ]:
plt.subplots(figsize=(9,9))
dft.groupby("Category")["Gross Sales"].sum().sort_values(ascending=False).head(10).plot(kind="pie",autopct="%.2f")

In [ ]:
dft["Created_month"]=dft["Created Date"].dt.month_name()

In [ ]:
dft[dft.Category=="Vegetables"].groupby("Created_month")['Gross Sales'].sum()

In [ ]:
dft["Created_week"] = dft["Created Date"].dt.week

In [ ]:
# Removing 27th week because there is not enough data points
dft = dft[dft.Created_week!=27]

In [ ]:
dft.groupby(by=['Created_week'],as_index=False)['Gross Sales'].count()

In [ ]:
dft_final = dft.groupby(by=['Created_week'],as_index=False)['Gross Sales'].sum()

In [ ]:
dft_final = dft_final.sort_values(by=['Created_week'],ascending=True)

In [ ]:
# dft_final["Created_week"] = dft_final["Created_week"].apply(lambda x: "week "+str(x))

In [ ]:
dft_final = dft_final.set_index("Created_week")

In [ ]:
plt.subplots(figsize=(20,7))
plt.plot(dft_final["Gross Sales"])
plt.title("Total Gross Sales")

In [ ]:
def rolling_statistics(dataframe):
    (plt.subplots(figsize=(20,7)))
    (plt.plot(dft_final.rolling(window=6).mean(),color='b',label='Mean'))
    (plt.plot(dft_final.rolling(window=6).std(),color='r',label='Standard deviation'))
    (plt.legend(loc='best'))
    (plt.xlabel("Week",color='k'))
    (plt.ylabel("Commercial TON",color='k'))
    plt.title("Rolling test statistics")

In [ ]:
def adf(dataframe_column):
    df_test = adfuller(dataframe_column,autolag='AIC')
    print("P-value for ADF test- ",df_test[1])
    if df_test[1] > 0.05:
        print("The data is not stationery")
    else:
        print("The data is stationery")

In [ ]:
def transformation(dataframe_column):
    return np.log(dataframe_column)

In [ ]:
rolling_statistics(dft_final)

In [ ]:
print("There is variance, data does not look stationery")

In [ ]:
# Checking auto-correlation
plot_acf(dft_final)

In [ ]:
adf(dft_final['Gross Sales'])

In [ ]:
splot=seasonal_decompose(dft_final,model='additive',freq=2)
splot.plot()

In [ ]:
# Below steps to make data stationery

In [ ]:
dft_logged = transformation(dft_final)

In [ ]:
dft_diff = dft_logged.diff(axis = 0, periods = 1).dropna()
# Differencing will make first week data 'nan - not a number'

In [ ]:
adf(dft_diff)

In [ ]:
dft_diff.shape

In [ ]:
# below steps to split data into test and train

In [ ]:
df_train = dft_diff[0:20]
df_test = dft_diff[20:]

In [ ]:
# below steps to find best parameters for model

In [ ]:
p=i=q=range(0,5)
piq = itertools.product(p,i,q)
piq=list(piq)

In [ ]:
for param in piq:
    try:
        model = ARIMA(df_train,order=param)
        results_AR = model.fit(disp=-1)
        print(param,results_AR.aic)
    except:
        continue
#checking the combination in which is error is less
#arima=it is a linera line with combination of auto regression and moving avg., it is one of the best model

In [ ]:
# below we are fitting the model

In [ ]:
model = ARIMA(df_train,order=(4, 0, 1))
results_AR = model.fit()
print(param,results_AR.aic)

#using 4,4,4 bcz is fits the model

In [ ]:
print("Printing predictions")
results_AR.predict(start=20,end=27)

In [ ]:
plt.plot(df_test,color="r")
plt.plot(results_AR.predict(start=20,end=27),color='g')
plt.title("Test data vs prediction")
plt.legend(["Test data", "predicted data"], loc ="top right")

In [ ]:
results_AR.plot_predict(1,25)

In [ ]:
# Following code normalizes the test data (predicted data)

In [ ]:
cols = dft_logged.columns
x = []
for col in cols:
    diff_results = dft_logged[col] + results_AR.predict(start=1,end=26).shift(-1)
    x.append(diff_results)
diff_df_inverted = pd.concat(x, axis=1)


In [ ]:
predicted_dataframe = np.exp(diff_df_inverted[:-1]).astype(str)

In [ ]:
predicted_dataframe = predicted_dataframe.rename(columns={0:"Gross Sales"})

In [ ]:
predicted_dataframe['Gross Sales'] = predicted_dataframe['Gross Sales'].str.split(".",expand=True)[0]

In [ ]:
predicted_dataframe[21:]

In [ ]:
dfl=dft.groupby(by=['Location Name'],as_index=False)['Gross Sales'].sum()
dfl